In [5]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.optimizers import Adam

In [10]:

def rotndup_img (img):
    img90 = np.array(list(zip(*img[::-1])))
    img180 = np.array(list(zip(*img90[::-1])))
    img270 = np.array(list(zip(*img180[::-1])))
    return [img, img90, img180, img270]


def deep_clean (bd1):
    # begins as a regular cleaning of noise anything below mean + 1 std will zero
    bd1 = np.array(bd1)
    mn, mx = bd1.min(), bd1.max()
    bd1 = (bd1 - mn) / (mx - mn)
    bd1[bd1 < (bd1.mean() + bd1.std() * 1)] = 0
    bd1 = bd1.reshape(75, 75)

    # defining an array to hold "remember" our object pattern
    part_of = np.zeros((75, 75))

    # we are searching waterfall style only upside down, and so we extract the peaks and work our way down
    # until we reach the previous pixel marked as 0. this will define the major structurs in the image.
    # All the rest is squashed.
    top95 = np.percentile(bd1, 99.5)
    row, col = np.where(bd1 > top95)

    lst = [[-1, 1], [0, 1], [1, 1], [-1, 0], [1, 0], [-1, -1], [0, -1], [-1, -1]]

    for i in range(len(row)):
        x = row[i]
        y = col[i]
        cell2chk = [[x, y]]
        j = 0
        macktub = True
        while macktub:
            x, y = cell2chk[j]
            part_of[x, y] = 1
            for sx, sy in lst:
                if (x+sx) < 75 and (y+sy) < 75 and (x+sx) >= 0 and (y+sy) >= 0:
                    if bd1[x + sx, y + sy] != 0 and part_of[x + sx, y + sy] == 0:
                        part_of[x + sx, y + sy] = 1
                        cell2chk.append([x + sx, y + sy])
                    elif bd1[x + sx, y + sy] == 0:
                        part_of[x + sx, y + sy] = -1
            j += 1
            if j + 1 >= len(cell2chk):
                macktub = False

    bd1[part_of == 0] = 0
    return bd1


def getModel():
    # Build keras model

    model = Sequential()

    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    model.add(Dropout(0.2))

    # CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    # Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    # Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(lr= 0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
 #   optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.01, decay=0.0, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [7]:

# load the training data
train = pd.read_json('/Users/ohad/Google_Drive/DS_projects/K_Statoil/Train/train.json')
train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce')

# option1 fill the images with nan for inc_angle with mean of angles
#train['inc_angle'] = train['inc_angle'].fillna(train['inc_angle'].mean())

# option2 drop images with nan
train = train.dropna(axis=0, how='any')

print('train has been read')


train has been read


In [11]:
# input image dimensions
img_rows, img_cols, layers = 75, 75, 1


# Creating more test cases
print('creating more test cases')

avg = 0
enh_ly = []
bnd1_ly = []
bnd2_ly = []
inc_ang_ly = []
inc_ang = np.zeros((75, 75))
X = []
y = []
for i in train.index:
    print(i)

    """"# cleaned layer band_1 HH (cHH)
    bnd1 = clean_img(train.at[i, 'band_1'])
    bnd1_ly.extend(rotndup_img(bnd1))

    # cleaned layer band_2 HV (in the future will cleaned based on the zero of HH) - cHV
    bnd2 = clean_img(train.at[i, 'band_2'])
    bnd2_ly.extend((rotndup_img(bnd2)))"""

    # cleaned layer band_1 HH (cHH)
    bnd1 = deep_clean(train.at[i, 'band_1'])
    bnd2 = np.array(train.at[i, 'band_2']).reshape(75, 75)
    bnd2[bnd1 == 0] = 0
    bnd1_ly.extend(rotndup_img(bnd1))

    # cleaned layer band_2 HV (in the future will cleaned based on the zero of HH) - cHV
    bnd2_ly.extend((rotndup_img(bnd2)))


    # Enhanced layer of cHH + cHV
    enh_img = bnd1 + bnd2
    avg = avg + np.mean(enh_img, axis = (0, 1))
    enh_ly.extend(rotndup_img(enh_img))

    # layer containing the angle
    inc_ang[bnd1 != 0] = train.at[i, 'inc_angle']
    inc_ang[bnd1 == 0] = 0
    inc_ang_ly.extend([inc_ang, inc_ang, inc_ang, inc_ang])

    # matching the target data
    ice = train.loc[i, 'is_iceberg']

    """
    # All 4 layers bnd1, bnd2, enh and angle
    X.append(np.dstack((bnd1_ly[0], bnd2_ly[0], enh_ly[0], inc_ang_ly[0])))
    X.append(np.dstack((bnd1_ly[1], bnd2_ly[1], enh_ly[1], inc_ang_ly[1])))
    X.append(np.dstack((bnd1_ly[2], bnd2_ly[2], enh_ly[2], inc_ang_ly[2])))
    X.append(np.dstack((bnd1_ly[3], bnd2_ly[3], enh_ly[3], inc_ang_ly[3])))

    y.extend([ice, ice, ice, ice])
    """

    """
    # layer NOT containing the angle and the enh_ly
    X.append(np.dstack((bnd1_ly[0], bnd2_ly[0])))
    X.append(np.dstack((bnd1_ly[1], bnd2_ly[1])))
    X.append(np.dstack((bnd1_ly[2], bnd2_ly[2])))
    X.append(np.dstack((bnd1_ly[3], bnd2_ly[3])))
    
    y.extend([ice, ice, ice, ice])
    """

    """
    # layer bnd1, bnd2, angle
    X.append(np.dstack((bnd1_ly[0], bnd2_ly[0], inc_ang_ly[0])))
    X.append(np.dstack((bnd1_ly[1], bnd2_ly[1], inc_ang_ly[1])))
    X.append(np.dstack((bnd1_ly[2], bnd2_ly[2], inc_ang_ly[2])))
    X.append(np.dstack((bnd1_ly[3], bnd2_ly[3], inc_ang_ly[3])))

    y.extend([ice, ice, ice, ice])
    """
    """
    # layer containing ONLY enh + angle with no duplication of the data
    X.append(np.dstack((enh_img, inc_ang)))

    y.extend([ice])
    """

    # layer containing ONLY enh with NO duplication of the data
    X.append(np.dstack(enh_img))
    
    y.extend([ice])

    """
    # layer containing ONLY enh with duplication of the data
    X.append(np.dstack(enh_ly[0]))
    X.append(np.dstack(enh_ly[1]))
    X.append(np.dstack(enh_ly[2]))
    X.append(np.dstack(enh_ly[3]))

    y.extend([ice, ice, ice, ice])
    """
"""
what I want to do here is create 5 layers:
    1. Cleaned layer band_1
    2. Cleaned layer band_2
    3. Enhanced using band_1 + band_2
    4. layer using the angle
    5. to be tested - img from 4 zoomed, sharpened 
"""

creating more test cases
0
1
2
3
4
5
6
7
8


9
10
11
12
13
14


15
16
17
18
19
20


21
22
23
24


25
26
27
28


...

1479
1480
1481
1482
1483
1484
1485
1486


1487
1488
1489
1490
1491
1492
1493


1494
1495
1496
1497
1498
1499
1500
1501
1502


1503
1504
1505
1506
1507
1508


'\nwhat I want to do here is create 5 layers:\n    1. Cleaned layer band_1\n    2. Cleaned layer band_2\n    3. Enhanced using band_1 + band_2\n    4. layer using the angle\n    5. to be tested - img from 4 zoomed, sharpened \n'

In [ ]:

X = np.array(X)
avg = avg/len(X)
X = X - avg
random.shuffle(X)
y = np.array(y)

# split train and validate data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# configuring the input_shape and the x, y to the tensor flow structure
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], layers, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], layers, img_rows, img_cols)
    input_shape = (layers, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, layers)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, layers)
    input_shape = (img_rows, img_cols, layers)


# change train to float32 for better performance
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Setup model parameters
batch_size = 100
epochs = 50
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

model = getModel()
model.summary()


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=[earlyStopping, mcp_save, reduce_lr_loss],
          validation_data=(x_test, y_test))

model.load_weights(filepath = '.mdl_wts.hdf5')

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

